<a href="https://colab.research.google.com/github/skotagiri95/BigData_Amazon/blob/master/Amazon_BigData1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, Findspark and download a Postgresql driver
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

!wget -q https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz
!gunzip amazon_reviews_us_PC_v1_00.tsv.gz

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

gzip: amazon_reviews_us_PC_v1_00.tsv already exists; do you wish to overwrite (y or n)? ^C


In [0]:
# Create a spark session, configured with Posetgres driver
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('AmazonETL')\
        .config("spark.driver.extraClassPath", "/content/postgresql-42.2.9.jar")\
        .getOrCreate()

In [0]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
pc_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
pc_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [0]:
# original dataset information
print((df.count(), len(df.columns)))
# check data types
df.printSchema

(5115666, 15)


<bound method DataFrame.printSchema of DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: string, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: timestamp]>

In [0]:
# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType

# Import struct fields that we can use
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DateType

In [0]:
schema = [StructField('marketplace', StringType(), True), StructField('customer_id', IntegerType(), True),
          StructField('review_id', StringType(), True), StructField('product_id', StringType(), True),
        
          StructField('product_category', StringType(), True), StructField('star_rating', IntegerType(), True),
          StructField('helpful_votes', IntegerType(), True), StructField('total_votes', IntegerType(), True),
          StructField('vine', StringType(), True), StructField('verified_purchase', StringType(), True),
          StructField('review_headline', StringType(), True), StructField('review_body', StringType(), True),
          StructField('review_date', DateType(), True), ]
schema

NameError: ignored

In [0]:
final=StructType(fields=schema)
me_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), schema=final, sep="\t", header=True)
me_df.printSchema()

In [0]:
# drop any rows with null or NaN
droped_df = casted_df.dropna()
droped_df.count()

# drop duplicate rows
droped_df.dropDuplicates().count()

In [0]:
review_id_table = clean_pc.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_table.show()

In [0]:
products=clean_pc.select(['product_id','product_title'])
products =products.dropDuplicates(['product_id'])
products.show()

In [0]:
customers = clean_pc.groupby('customer_id').count().select('customer_id', col('count').alias('customer_count'))
customers.show()

In [0]:
vine_table = clean_pc.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_table.show()

In [0]:
paid_vine_df = vine_table.filter(col('vine')=='Y')
paid_vine_df.orderBy(paid_vine_df['helpful_votes'].desc()).select('star_rating','helpful_votes','total_votes').show()

In [0]:
paid_vine_df.filter('star_rating = 5').select('helpful_votes').count()

In [0]:
paid_vine_avg = paid_vine_df.groupBy('star_rating').avg()
paid_vine_avg.orderBy(non_vine_avg['star_rating'].desc()).select('star_rating','avg(helpful_votes)','avg(total_votes)').show()

In [0]:
paid_vine_df.select('review_id').count()

In [0]:
paid_vine_df.select('helpful_votes').count()

In [0]:
# Configure setting for RDS
mode = 'append'
jdbc_url="jdbc:postgresql://dataviz.cbp360boojkd.us-east-2.rds.amazonaws.com:5431/AWS_PC_Reviews"
config = {"user":"skotagiri95",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [0]:
#Write DataFrame to review_id_table table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

# Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

# Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)